# Loading the Open COVID-19 Dataset
This very short notebook showcases how to load the [Open COVID-19 datset](https://github.com/open-covid-19/data), including some examples for commonly performed operations.

First, loading the data is very simple with `pandas`. We can use the CSV or the JSON file to download the entire Open COVID-19 dataset in a single step:

In [1]:
import pandas as pd

# Load CSV data directly from the URL with pandas
data = pd.read_csv('https://open-covid-19.github.io/data/v2/master.csv')

# Alternatively load the JSON data, which should be identical
data_json = pd.read_json('https://open-covid-19.github.io/data/v2/master.json')
assert len(data) == len(data_json)

# Print a small snippet of the dataset
print('The dataset currently contains %d records, here are the last few:' % len(data))
data.tail()

The dataset currently contains 268684 records, here are the last few:


,key,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,3166-1-alpha-2,...,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index
268679,ZW,Q954,country/ZWE,ZW,Zimbabwe,NaN,NaN,NaN,NaN,ZW,...,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,91.4
268680,ZW,Q954,country/ZWE,ZW,Zimbabwe,NaN,NaN,NaN,NaN,ZW,...,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,91.4
268681,ZW,Q954,country/ZWE,ZW,Zimbabwe,NaN,NaN,NaN,NaN,ZW,...,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,91.4
268682,ZW,Q954,country/ZWE,ZW,Zimbabwe,NaN,NaN,NaN,NaN,ZW,...,1.0,NaN,NaN,NaN,2.0,1.0,1.0,0.0,0.0,NaN
268683,ZW,Q954,country/ZWE,ZW,Zimbabwe,NaN,NaN,NaN,NaN,ZW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Looking at country-level data
Some records contain country-level data, in other words, data that is aggregated at the country level. Other records contain region-level data, which are subdivisions of a country; for example, Chinese provinces or USA states. A few regions also report at an even smaller subdivision, i.e. county/municipality level.

To filter only country-level data from the dataset, look for records that have a `aggregation_level == 0` or, alternatively, null value for the `subregion1_code` (or `subregion1_name`) field:

In [2]:
# Look for rows with country level data
# Same as `data[data.subregion2_code.isna()]`
countries = data[data.aggregation_level == 0]

# We no longer need the subregion-level columns
countries = countries.drop(columns=['subregion1_code', 'subregion1_name', 'subregion2_code', 'subregion2_name'])

countries.tail()

,key,wikidata,datacommons,country_code,country_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,date,new_confirmed,...,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index
268679,ZW,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,2020-05-16,2.0,...,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,91.4
268680,ZW,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,2020-05-17,2.0,...,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,91.4
268681,ZW,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,2020-05-18,0.0,...,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,91.4
268682,ZW,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,2020-05-19,0.0,...,1.0,NaN,NaN,NaN,2.0,1.0,1.0,0.0,0.0,NaN
268683,ZW,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,2020-05-20,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Looking at state/province data
Conversely, to filter state/province data for a specific country, we need to look for records where the aggregation level is `1` (or where the region columns have non-null values). The following snippet extracts data related to Spain's subregions from the dataset:

In [3]:
# Filter records that have the right country code AND a non-null region code
# Same as `data[(data.country_code == 'ES') & ~(data.subregion`_code.isna())]`
spain_regions = data[(data.country_code == 'ES') & (data.aggregation_level == 1)]

# We no longer need the municipality-level columns
spain_regions = spain_regions.drop(columns=['subregion2_code', 'subregion2_name'])

spain_regions.tail()

,key,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,...,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index
23362,ES_VC,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23363,ES_VC,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23364,ES_VC,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23365,ES_VC,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23366,ES_VC,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Using the `key` column
The `key` column is present in all datasets and is unique for each combination of country, province/state and municipality/county. This way, we can retrieve a specific country or region using a single filter for the data. The `key` column is built using `country_code` for country-level data, `${country_code}_${subregion1_code}` for province/state level data, and `${country_code}_${subregion1_code}_${subregion2_code}` for municipality/county data:

In [5]:
# Filter records for Spain at the country-level
spain_country = data[data.key == 'ES']

# We no longer need the subregion-level columns
spain_country = spain_country.drop(columns=['subregion1_code', 'subregion1_name', 'subregion2_code', 'subregion2_name'])

spain_country.tail()

,key,wikidata,datacommons,country_code,country_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,date,new_confirmed,...,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index
21652,ES,Q29,country/ESP,ES,Spain,ES,ESP,0,2020-05-16,562.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21653,ES,Q29,country/ESP,ES,Spain,ES,ESP,0,2020-05-17,321.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21654,ES,Q29,country/ESP,ES,Spain,ES,ESP,0,2020-05-18,209.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21655,ES,Q29,country/ESP,ES,Spain,ES,ESP,0,2020-05-19,416.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21656,ES,Q29,country/ESP,ES,Spain,ES,ESP,0,2020-05-20,518.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Filter records for Madrid, one of the subregions of Spain
madrid = data[data.key == 'ES_MD']

madrid.tail()

,key,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,3166-1-alpha-2,...,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index
22912,ES_MD,Q5756,nuts/ES30,ES,Spain,MD,Madrid,NaN,NaN,ES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22913,ES_MD,Q5756,nuts/ES30,ES,Spain,MD,Madrid,NaN,NaN,ES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22914,ES_MD,Q5756,nuts/ES30,ES,Spain,MD,Madrid,NaN,NaN,ES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22915,ES_MD,Q5756,nuts/ES30,ES,Spain,MD,Madrid,NaN,NaN,ES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22916,ES_MD,Q5756,nuts/ES30,ES,Spain,MD,Madrid,NaN,NaN,ES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dataset Subsets
The master table can be large and cumbersome depending on your application. If you only need a subset of the data, for example only epidemiology, here's how you would get only that data for Madrid:

In [7]:
# Load the epidemiology table
# Note that all the helper columns such as country code, country name, aggregation level, etc. are present in the
# `index` table; we only have the key here
epi = pd.read_csv('https://open-covid-19.github.io/data/v2/epidemiology.csv')

# Filter records for Madrid, one of the subregions of Spain
madrid = epi[epi.key == 'ES_MD']

madrid.tail()

,date,key,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested
243091,2020-05-15,ES_MD,104.0,17.0,NaN,NaN,66573.0,8826.0,NaN,NaN
247340,2020-05-16,ES_MD,24.0,21.0,NaN,NaN,66597.0,8847.0,NaN,NaN
251534,2020-05-17,ES_MD,28.0,16.0,NaN,NaN,66625.0,8863.0,NaN,NaN
255749,2020-05-18,ES_MD,123.0,31.0,NaN,NaN,66748.0,8894.0,NaN,NaN
259962,2020-05-19,ES_MD,112.0,18.0,NaN,NaN,66860.0,8912.0,NaN,NaN


### Data consistency
Often, region-level data and country-level data will come from different sources. This will lead to numbers not adding up exactly, or even date misalignment (the data for the region may be reported sooner or later than the whole country). However, country- and region- level data will *always* be self-consistent